In [5]:
#library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as ap

In [6]:
#mount file from google drive
from google.colab import drive
drive.mount('/content/drive')
#grab the data
img224 = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/img224.npy')
label  = np.load('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/label.npy')

Mounted at /content/drive


In [7]:
X224_train = img224
y224_train = label
y224_train = tf.keras.utils.to_categorical(y224_train, 4)

In [8]:
#checkpoint
checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/FINAL_DL/checkpoint/vgg2'
model_checkpoint_callack = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weight_only = True,
    save_freq = 'epoch',
    mode = 'auto',
    save_best_only = True,
    monitor = 'val_accuracy'
)

In [9]:
#VGG16
def model_vgg16(X_train, y_train,epochs = 8):
    base_model = ap.VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
    base_model.trainable = False

    head_model = base_model.output
    head_model = keras.layers.Flatten()(head_model)
    #head_model = keras.layers.Dense(512, activation = 'relu')(head_model)
    #head_model = keras.layers.Dropout(0.5)(head_model)
    #head_model = keras.layers.Dense(128, activation = 'relu')(head_model)
    head_model = keras.layers.Dense(4,activation='softmax')(head_model)

    model = keras.Model(base_model.input,head_model)

    callbacks = [keras.callbacks.TensorBoard(log_dir='/content/drive/MyDrive/Colab Notebooks/FINAL_DL/logs/vgg2'),model_checkpoint_callack]
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train,y_train,batch_size=32, epochs=epochs, callbacks = callbacks, validation_split = 0.2)

    return model,history

In [11]:
model_vgg16_m2,history= model_vgg16(X224_train,y224_train,16)

Epoch 1/16
75/75 [==============================] - 96s 1s/step - loss: 0.7968 - accuracy: 0.7067 - val_loss: 0.4590 - val_accuracy: 0.8400
Epoch 2/16
75/75 [==============================] - 96s 1s/step - loss: 0.3049 - accuracy: 0.8938 - val_loss: 0.3381 - val_accuracy: 0.8767
Epoch 3/16
75/75 [==============================] - 98s 1s/step - loss: 0.2054 - accuracy: 0.9362 - val_loss: 0.3023 - val_accuracy: 0.9017
Epoch 4/16
75/75 [==============================] - 94s 1s/step - loss: 0.1546 - accuracy: 0.9558 - val_loss: 0.3001 - val_accuracy: 0.9017
Epoch 5/16
75/75 [==============================] - 96s 1s/step - loss: 0.1213 - accuracy: 0.9642 - val_loss: 0.2576 - val_accuracy: 0.9167
Epoch 6/16
75/75 [==============================] - 91s 1s/step - loss: 0.0831 - accuracy: 0.9875 - val_loss: 0.2685 - val_accuracy: 0.8983
Epoch 7/16
75/75 [==============================] - 91s 1s/step - loss: 0.0711 - accuracy: 0.9883 - val_loss: 0.2366 - val_accuracy: 0.9083
Epoch 8/16
75/75 [==

In [12]:
model_vgg16_m2.save_weights('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/weights/vgg_m2_weights.h5')

In [15]:
#save the curve
his = pd.DataFrame(history.history)
his.to_pickle('/content/drive/MyDrive/Colab Notebooks/FINAL_DL/curve/history_vgg2.pkl')

In [14]:
model_vgg16_m2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   